In [2]:
import tensorflow as tf
import os
import numpy as np
import cv2
from tensorflow.keras.layers import Layer

In [3]:
# Siamese L1 Distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [33]:
# Reload model 
model = tf.keras.models.load_model('siamesemodel2.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [4]:
# model.summary()

In [34]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

In [53]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [45]:
results, verified = verify(model, 0.5, 0.5)
print(verified)

1/1 [==============================] - 1s 605ms/step
True


In [54]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(model, 0.5, 0.5)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 581ms/step
True


In [51]:
np.sum(np.squeeze(results) > 0.5)

0

In [30]:
44/44

1.0

In [49]:
results

[array([[3.2012384e-08]], dtype=float32),
 array([[2.6111556e-08]], dtype=float32),
 array([[2.3469413e-08]], dtype=float32),
 array([[2.2952698e-08]], dtype=float32),
 array([[2.0918847e-08]], dtype=float32),
 array([[2.6219103e-08]], dtype=float32),
 array([[3.468118e-08]], dtype=float32),
 array([[2.7152549e-08]], dtype=float32),
 array([[2.7781507e-08]], dtype=float32),
 array([[3.8490843e-08]], dtype=float32),
 array([[2.3275405e-08]], dtype=float32),
 array([[3.9547967e-08]], dtype=float32),
 array([[4.7268752e-08]], dtype=float32),
 array([[2.3310013e-08]], dtype=float32),
 array([[3.8155118e-08]], dtype=float32),
 array([[2.4838146e-08]], dtype=float32),
 array([[4.0608246e-08]], dtype=float32),
 array([[2.6056082e-08]], dtype=float32),
 array([[4.074223e-08]], dtype=float32),
 array([[4.021453e-08]], dtype=float32),
 array([[4.7331273e-08]], dtype=float32),
 array([[3.00915e-08]], dtype=float32),
 array([[3.913765e-08]], dtype=float32),
 array([[3.182026e-08]], dtype=float32),

In [ ]:
x